In [15]:
import qpsolvers
from os.path import exists
import pickle

import sn_bayes
from sn_bayes import longevity_bayes
from sn_bayes.utils import complexity_check
from sn_bayes.utils import get_var_positions
from sn_bayes.utils import get_var_val_positions
from sn_bayes.utils import make_tree
from sn_bayes.utils import bayesInitialize
from sn_bayes.utils import query
from sn_bayes.utils import internal_query
from sn_bayes.utils import internal_query
from sn_bayes.utils import explain_why_bad
from sn_bayes.utils import explain_why_good
from sn_bayes.utils import create_query
from sn_bayes.utils import dependency
from sn_bayes.utils import addCpt
from sn_bayes.utils import non_cpt_descriptions
from sn_service.service_spec.bayesian_pb2 import BayesianNetwork

import sn_service.service_spec.bayesian_pb2
from sn_service.service_spec.bayesian_pb2 import Query
import sn_service.service_spec.bayesian_pb2_grpc as grpc_bayes_grpc
import sn_service.service_spec.bayesian_pb2
from sn_service.service_spec.bayesian_pb2 import BayesianNetworkQuery
from sn_service.service_spec.bayesian_pb2 import QueryId
from sn_service.service_spec.bayesian_pb2 import Id

import grpc
import pandas as pd
import networkx as nx
import time
import re

import matplotlib.pyplot as plt

In [21]:
bayesianNetwork = BayesianNetwork()
cpt = {}
outstr = ''
outstr = outstr + addCpt(bayesianNetwork,cpt) 

cpt["cardiovascular_disease"] = dependency(bayesianNetwork,cpt, 
    [
    ({"age":["elderly"]},{"relative_risk":7}),
    ({"diabetes":["diabetes"]},{"relative_risk":3}),
    ({"obesity":["obesity"]},{"relative_risk":2}),
    ({"heart_disorder_indicators":["heart_disorder_indicators"]},{"sensitivity":0.4, "specificity":0.3}),
    ({"hypertension":["hypertension"]},{"relative_risk":3.15})
            
    ],
    {"cardiovascular_disease":0.09,"no_cardiovascular_disease":0.91}
    )

start timing...
start timing...


IndexError: list index out of range

In [23]:
discreteDistribution = bayesianNetwork.discreteDistributions.add()
discreteDistribution.name = "cough"
variable = discreteDistribution.variables.add()
variable.name = "cough_up_blood"
variable.probability = 0.01
variable = discreteDistribution.variables.add()
variable.name = "cough_with_green_phlegm"
variable.probability = 0.04
variable = discreteDistribution.variables.add()
variable.name = "cough_with_clear_spitum"
variable.probability = 0.05
variable = discreteDistribution.variables.add()
variable.name = "no_cough"
variable.probability = 0.90

In [24]:
cpt["cardiovascular_disease"] = dependency(bayesianNetwork,cpt, 
    [
    ({"age":["elderly"]},{"relative_risk":7}),
    ({"diabetes":["diabetes"]},{"relative_risk":3}),
    ({"obesity":["obesity"]},{"relative_risk":2}),
    ({"heart_disorder_indicators":["heart_disorder_indicators"]},{"sensitivity":0.4, "specificity":0.3}),
    ({"hypertension":["hypertension"]},{"relative_risk":3.15})
            
    ],
    {"cardiovascular_disease":0.09,"no_cardiovascular_disease":0.91}
    )

start timing...
start timing...


KeyError: 'age'